<a href="https://colab.research.google.com/github/Udayprofile/Udayrep/blob/main/SALESDATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I am here Creating a Sales Data Analysis Project. Where, I am Going to generate sample sales data, clean it, and Analyse trends using PySpark and SQL


Importing and installing the essential libraries required.


In [14]:
!pip install pyspark
import pandas as pd
import random
from datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, count, to_date


Creating a Spark session




In [5]:
spark = SparkSession.builder.appName("SalesDataAnalysis").getOrCreate()


Generating Synthetic Sales Data

I am creating here 1000 random sales records with columns:

Order_ID: Unique order number

Date: Random date in the last 6

Product: Random product name

Category: Product category

Quantity: Random quantity (1-10)

Price: Random price (₹100-₹1000)

Total_Sale: Quantity × Price


In [6]:
# Generate random sales data
products = ["Laptop", "Mobile", "Tablet", "Smartwatch", "Headphones", "Camera"]
categories = {"Laptop": "Electronics", "Mobile": "Electronics", "Tablet": "Electronics",
              "Smartwatch": "Wearable", "Headphones": "Accessories", "Camera": "Photography"}

sales_data = []
start_date = datetime.today() - timedelta(days=180)  # Last 6 months

for i in range(1000):
    date = start_date + timedelta(days=random.randint(0, 180))
    product = random.choice(products)
    category = categories[product]
    quantity = random.randint(1, 10)
    price = random.randint(100, 1000)
    total_sale = quantity * price
    sales_data.append([i+1, date.strftime('%Y-%m-%d'), product, category, quantity, price, total_sale])

# Convert to Pandas DataFrame
columns = ["Order_ID", "Date", "Product", "Category", "Quantity", "Price", "Total_Sale"]
sales_df = pd.DataFrame(sales_data, columns=columns)

# Convert Pandas DF to Spark DF
spark_df = spark.createDataFrame(sales_df)

# Show first few rows
spark_df.show(5)


+--------+----------+----------+-----------+--------+-----+----------+
|Order_ID|      Date|   Product|   Category|Quantity|Price|Total_Sale|
+--------+----------+----------+-----------+--------+-----+----------+
|       1|2024-10-17|Headphones|Accessories|       2|  577|      1154|
|       2|2024-12-31|    Tablet|Electronics|      10|  573|      5730|
|       3|2025-02-11|    Mobile|Electronics|       9|  279|      2511|
|       4|2024-11-02|    Laptop|Electronics|       5|  480|      2400|
|       5|2024-12-02|Smartwatch|   Wearable|       9|  632|      5688|
+--------+----------+----------+-----------+--------+-----+----------+
only showing top 5 rows



Data Cleaning
Converting the Date column to proper date format
Remove duplicate records (if any)

In [7]:
spark_df = spark_df.withColumn("Date", to_date(col("Date")))
spark_df = spark_df.dropDuplicates()
spark_df.printSchema()


root
 |-- Order_ID: long (nullable = true)
 |-- Date: date (nullable = true)
 |-- Product: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Price: long (nullable = true)
 |-- Total_Sale: long (nullable = true)



Doing SQL Queries in PySpark
Registeing DataFrame as SQL Table:

In [8]:
spark_df.createOrReplaceTempView("sales_data")


Total Sales by Product

In [9]:
spark.sql("SELECT Product, SUM(Total_Sale) as TotalRevenue FROM sales_data GROUP BY Product ORDER BY TotalRevenue DESC").show()


+----------+------------+
|   Product|TotalRevenue|
+----------+------------+
|    Camera|      568832|
|    Mobile|      558369|
|    Laptop|      525258|
|Smartwatch|      468205|
|Headphones|      453828|
|    Tablet|      395601|
+----------+------------+



Average sales per Category

In [10]:
spark.sql("SELECT Category, AVG(Total_Sale) as AvgSale FROM sales_data GROUP BY Category").show()


+-----------+------------------+
|   Category|           AvgSale|
+-----------+------------------+
|   Wearable|3060.1633986928105|
|Electronics| 3006.560975609756|
|Accessories|2801.4074074074074|
|Photography| 2947.316062176166|
+-----------+------------------+



Most Sold products

In [11]:
spark.sql("SELECT Product, SUM(Quantity) as TotalUnitsSold FROM sales_data GROUP BY Product ORDER BY TotalUnitsSold DESC LIMIT 5").show()


+----------+--------------+
|   Product|TotalUnitsSold|
+----------+--------------+
|    Camera|          1055|
|    Mobile|           969|
|    Laptop|           939|
|Headphones|           854|
|Smartwatch|           844|
+----------+--------------+



SAVING the results

In [12]:
spark_df.write.csv("sales_data_cleaned.csv", header=True)
